In [1]:
from Pipelines.process import *
import matplotlib.pyplot as plt
import seaborn as sns
from pygam import LinearGAM, GAM, s, f, te, l
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import re
from sklearn.model_selection import GridSearchCV
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
import math
import numpy as np

In [2]:
df_1 = process_raw("Data/train.csv").pipe(take_difference).pipe(take_population_rates).pipe(imputation_waves)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_252"; OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09); OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
  Starting server from /home/datageek/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpj6hvo7ra
  JVM stdout: /tmp/tmpj6hvo7ra/h2o_datageek_started_from_python.out
  JVM stderr: /tmp/tmpj6hvo7ra/h2o_datageek_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_datageek_eeu75d
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.89 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


100% (300 of 300) |######################| Elapsed Time: 0:02:13 Time:  0:02:13
100% (300 of 300) |######################| Elapsed Time: 0:02:20 Time:  0:02:20
100% (300 of 300) |######################| Elapsed Time: 0:02:16 Time:  0:02:16
100% (300 of 300) |######################| Elapsed Time: 0:01:21 Time:  0:01:21


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Grid Build progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Grid Build progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/datageek/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'galaxy' has levels not trained on: [KKR 03 (KK98 230)]
  warnings.warn(w)


...

gbm prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Grid Build progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


/home/datageek/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'galaxy' has levels not trained on: [Sextans A (92205, DDO 75), UGC 8651 (DDO 181), UGCA 438 (ESO 407-018)]
  warnings.warn(w)


gbm prediction progress: |████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Grid Build progress: |████████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


/home/datageek/.local/lib/python3.6/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'galaxy' has levels not trained on: [Andromeda XI, Andromeda XII, Andromeda XVIII[60], ESO 383-087 (ISG 39), Eridanus II, IC 342, KKh 086, NGC 4214 (UGC 7278), UGCA 86]
  warnings.warn(w)


drf prediction progress: |████████████████████████████████████████████████| 100%
H2O session _sid_a0a5 closed.


In [3]:
df_1.to_csv("Data/train_imputed.csv")